In [1]:
import pandas as pd

In [2]:
file_path = "results/model-validation/FP32.csv"
df32 = pd.read_csv(file_path)
df32.head()

,image,gt_x1,gt_y1,gt_x2,gt_y2,pr_x1,pr_y1,pr_x2,pr_y2,pr_score,IoU
0,img_20230322_224503.jpg,3659,2635,3777,2990,3661.0,2638.0,3809.0,2901.0,0.776861,0.606449
1,img_20230708_223004.jpg,2751,2609,3111,2808,2746.0,2611.0,3036.0,2806.0,0.876259,0.765338
2,img_20230624_014503.jpg,3121,731,3299,1048,3113.0,730.0,3307.0,1059.0,0.902837,0.884060
3,img_20230718_001503.jpg,3366,2088,3643,2354,3370.0,2097.0,3622.0,2356.0,0.862179,0.872995
4,img_20230616_224503.jpg,1288,2412,1539,2606,1287.0,2418.0,1540.0,2613.0,0.893374,0.928149


In [3]:
# Contar las filas que contienen al menos un NaN
num_rows_with_nan = df32.isna().any(axis=1).sum()
print(f"Número de filas con al menos un valor NaN: {num_rows_with_nan} de {df32.shape[0]}")
# Calcular el porcentaje de filas donde hay NaN
percentage = (num_rows_with_nan / df32.shape[0]) * 100
print(f"Porcentaje de filas con al menos un valor NaN: {percentage:.2f}%")

Número de filas con al menos un valor NaN: 2 de 147
Porcentaje de filas con al menos un valor NaN: 1.36%


In [4]:
# Contar las filas donde la columna 'IoU' es mayor o igual a 0.5
num_rows_iou_ge_0_5 = (df32['IoU'] >= 0.5).sum()
print(f"Número de filas donde 'IoU' es mayor o igual a 0.5: {num_rows_iou_ge_0_5} de {df32.shape[0]}")


# Calcular el porcentaje de filas donde 'IoU' es mayor o igual a 0.5
percentage = (num_rows_iou_ge_0_5 / df32.shape[0]) * 100
print(f"Porcentaje de filas donde 'IoU' es mayor o igual a 0.5: {percentage:.2f}%")

Número de filas donde 'IoU' es mayor o igual a 0.5: 141 de 147
Porcentaje de filas donde 'IoU' es mayor o igual a 0.5: 95.92%


## FP16

In [5]:
file_path = "results/model-validation/FP16.csv"
df16 = pd.read_csv(file_path)
df16.head()

,image,gt_x1,gt_y1,gt_x2,gt_y2,pr_x1,pr_y1,pr_x2,pr_y2,pr_score,IoU
0,img_20230322_224503.jpg,3659,2635,3777,2990,3662.0,2639.0,3810.0,2902.0,0.776977,0.598094
1,img_20230708_223004.jpg,2751,2609,3111,2808,2747.0,2612.0,3037.0,2807.0,0.876298,0.770091
2,img_20230624_014503.jpg,3121,731,3299,1048,3114.0,731.0,3308.0,1060.0,0.902805,0.884060
3,img_20230718_001503.jpg,3366,2088,3643,2354,3371.0,2097.0,3623.0,2356.0,0.862200,0.872995
4,img_20230616_224503.jpg,1288,2412,1539,2606,1287.0,2418.0,1539.0,2613.0,0.893417,0.931722


In [6]:
# Contar las filas que contienen al menos un NaN
num_rows_with_nan = df16.isna().any(axis=1).sum()
print(f"Número de filas con al menos un valor NaN: {num_rows_with_nan} de {df16.shape[0]}")
# Calcular el porcentaje de filas donde hay NaN
percentage = (num_rows_with_nan / df16.shape[0]) * 100
print(f"Porcentaje de filas con al menos un valor NaN: {percentage:.2f}%")

Número de filas con al menos un valor NaN: 2 de 147
Porcentaje de filas con al menos un valor NaN: 1.36%


In [7]:
# Contar las filas donde la columna 'IoU' es mayor o igual a 0.5
num_rows_iou_ge_0_5 = (df16['IoU'] >= 0.5).sum()
print(f"Número de filas donde 'IoU' es mayor o igual a 0.5: {num_rows_iou_ge_0_5} de {df16.shape[0]}")


# Calcular el porcentaje de filas donde 'IoU' es mayor o igual a 0.5
percentage = (num_rows_iou_ge_0_5 / df16.shape[0]) * 100
print(f"Porcentaje de filas donde 'IoU' es mayor o igual a 0.5: {percentage:.2f}%")

Número de filas donde 'IoU' es mayor o igual a 0.5: 141 de 147
Porcentaje de filas donde 'IoU' es mayor o igual a 0.5: 95.92%
